In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv("../input/stumbleupon/train.tsv", delimiter='\t')
test=pd.read_csv("../input/stumbleupon/test.tsv",delimiter='\t')

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.linear_model as lm
import pandas as p
from sklearn.model_selection import cross_val_score


loadData = lambda f: np.genfromtxt(open(f,'r'), delimiter=' ')

def main():

  print ("loading data..")
  traindata = list(np.array(p.read_table('../input/stumbleupon/train.tsv'))[:,2])
  testdata = list(np.array(p.read_table('../input/stumbleupon/test.tsv'))[:,2])
  y = np.array(p.read_table('../input/stumbleupon/train.tsv'))[:,-1]  
  y=y.astype('int')   

  tfv = TfidfVectorizer(min_df=3,  max_features=None, strip_accents='unicode',  
        analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)

  rd = lm.LogisticRegression(penalty='l2', dual=False, tol=0.0001, 
                             C=1, fit_intercept=True, intercept_scaling=1.0, 
                             class_weight=None, random_state=None)

  X_all = traindata + testdata
  lentrain = len(traindata)

  print ("fitting pipeline")
  tfv.fit(X_all)
  print ("transforming data")
  X_all = tfv.transform(X_all)

  X = X_all[:lentrain]
  X_test = X_all[lentrain:]

  print ("20 Fold CV Score: ", np.mean(cross_val_score(rd, X, y, cv=20, scoring='roc_auc')))

  print ("training on full data")
  rd.fit(X,y)
  pred = rd.predict_proba(X_test)[:,1]
  testfile = p.read_csv('../input/stumbleupon/test.tsv', sep="\t", na_values=['?'], index_col=1)
  pred_df = p.DataFrame(pred, index=testfile.index, columns=['label'])
  pred_df.to_csv('benchmark.csv')
  print ("submission file created..")

if __name__=="__main__":
  main()
